In [1]:
%load_ext autoreload

In [2]:
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer

In [7]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

import meshcat

# Build plant

In [4]:

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

simple_collision = True
directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()
# visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, 
#                                       delete_prefix_on_load=False)

# diagram = builder.Build()
# visualizer.load()


# Setup meshcat visualization

In [5]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph)
diagram = visualizer.diagram

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


# Run SNOPT IRIS in rational space

In [20]:
iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 5
iris_options.configuration_space_margin = 1e-5
iris_options.termination_threshold = -1
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False
# uncomment to test if non-default is working
# currently throwing std::bad_alloc if we try to set q_star
iris_options.q_star = 0.5 * np.zeros(3)
rational_forward_kinematics = RationalForwardKinematics(plant)


#seed points specified in q space
seed_points_q = np.array([
                        [0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]      # out of shelf 1
                        ]) 
seed_points_t = np.array([rational_forward_kinematics.ComputeTValue(s, iris_options.q_star) for s in seed_points_q])


In [21]:
regions = []
context = diagram.CreateDefaultContext()
for i, s in enumerate(seed_points_q):
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
    if i > 0:
        starting_hpolyhedron = regions[i-1]
        r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
                                              iris_options, starting_hpolyhedron)
    else:
        r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
    regions.append(r)
    print(f'Completed region: {i+1}/{len(seed_points)}')

Completed region: 1/6
Completed region: 2/6
Completed region: 3/6
Completed region: 4/6
Completed region: 5/6
Completed region: 6/6


In [22]:
# plot regions and collision constraint
if do_viz:
    visualizer.plot_regions(regions)
    visualizer.plot_seedpoints(seed_points_t)
    visualizer.visualize_collision_constraint(N = 50)

# Begin Certification

In [28]:
from pydrake.all import CspaceFreeRegion, SeparatingPlaneOrder, CspaceRegionType
cspace_free_region = CspaceFreeRegion(diagram, plant, scene_graph, 
                                      SeparatingPlaneOrder.kAffine, 
                                      CspaceRegionType.kGenericPolytope)

query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(diagram.CreateDefaultContext()))
inspector = query.inspector()
pairs = inspector.GetCollisionCandidates()

TypeError: GenerateTuplesForBilinearAlternation(): incompatible function arguments. The following argument types are supported:
    1. (self: pydrake.multibody.rational_forward_kinematics.CspaceFreeRegion, q_star: numpy.ndarray[numpy.float64[m, 1]], filtered_collision_pairs: Set[drake::SortedPair<drake::geometry::GeometryId>], C_rows: int) -> Tuple[List[pydrake.multibody.rational_forward_kinematics.CspaceFreeRegion.CspacePolytopeTuple], numpy.ndarray[object[m, 1]], numpy.ndarray[numpy.float64[m, 1]], numpy.ndarray[numpy.float64[m, 1]], numpy.ndarray[object[m, 1]], numpy.ndarray[object[m, 1]], numpy.ndarray[object[m, n]], numpy.ndarray[object[m, 1]], numpy.ndarray[object[m, 1]], numpy.ndarray[object[m, 1]], numpy.ndarray[object[m, 1]]]

Invoked with: <pydrake.multibody.rational_forward_kinematics.CspaceFreeRegion object at 0x7fc9ab0a00f0>, array([0., 0., 0.]), {(<GeometryId value=71>, <GeometryId value=113>), (<GeometryId value=71>, <GeometryId value=91>), (<GeometryId value=85>, <Geome

In [31]:
r = regions[0]
pair_set = set([*pairs])
r2 = cspace_free_region.CspacePolytopeBilinearAlternation(iris_options.q_star,
                                                    pairs,r.A(), r.b())

TypeError: CspacePolytopeBilinearAlternation(): incompatible function arguments. The following argument types are supported:
    1. (self: pydrake.multibody.rational_forward_kinematics.CspaceFreeRegion, q_star: numpy.ndarray[numpy.float64[m, 1]], filtered_collision_pairs: Set[drake::SortedPair<drake::geometry::GeometryId>], C_init: numpy.ndarray[numpy.float64[m, n], flags.f_contiguous], d_init: numpy.ndarray[numpy.float64[m, 1]], bilinear_alternation_option: pydrake.multibody.rational_forward_kinematics.BilinearAlternationOption, solver_options: pydrake.solvers.mathematicalprogram.SolverOptions) -> Tuple[numpy.ndarray[numpy.float64[m, n]], numpy.ndarray[numpy.float64[m, 1]], numpy.ndarray[numpy.float64[m, n]], numpy.ndarray[numpy.float64[m, 1]]]

Invoked with: <pydrake.multibody.rational_forward_kinematics.CspaceFreeRegion object at 0x7fc9ab0a00f0>, array([0., 0., 0.]), {(<GeometryId value=71>, <GeometryId value=113>), (<GeometryId value=71>, <GeometryId value=91>), (<GeometryId value=

In [33]:
pairs

{(<GeometryId value=71>, <GeometryId value=85>),
 (<GeometryId value=71>, <GeometryId value=91>),
 (<GeometryId value=71>, <GeometryId value=113>),
 (<GeometryId value=71>, <GeometryId value=115>),
 (<GeometryId value=71>, <GeometryId value=117>),
 (<GeometryId value=79>, <GeometryId value=113>),
 (<GeometryId value=79>, <GeometryId value=115>),
 (<GeometryId value=79>, <GeometryId value=117>),
 (<GeometryId value=85>, <GeometryId value=91>),
 (<GeometryId value=85>, <GeometryId value=113>),
 (<GeometryId value=85>, <GeometryId value=115>),
 (<GeometryId value=85>, <GeometryId value=117>),
 (<GeometryId value=91>, <GeometryId value=113>),
 (<GeometryId value=91>, <GeometryId value=115>),
 (<GeometryId value=91>, <GeometryId value=117>)}